In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import csv

In [2]:
# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.common.by import By

In [3]:

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [4]:
# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

Fase 2: Extracción de Detalles de Películas con Selenium
Una vez que tengan una lista de películas de la API, el siguiente paso es obtener información más detallada sobre ellas. Utilizaras la herramienta de automatización web Selenium para navegar a sitios web de reseñas de películas (IMDB y Rotten Tomatoes) y extraer detalles como calificaciones, actores y directores. En concreto deberás extraer información sobre:

Puntuacion de IMDB (en caso de que la tenga).

Puntuación de Rotten Tomatoes (Tomatometer).

Dirección (director/a o directore/as de cada película).

Gionistas (de cada película).

Argumento.

Duración (en minutos).

Nombre de la película

Probar un try para el buscador y en except break, asi me devuelve los resultados en csv hasta el momento?

In [6]:
# # Llamo al navegador

# driver=webdriver.Chrome()
# driver.get("https://www.imdb.com/?ref_=nv_home")
# driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()

# # Hago grande el navegador para que aparezcan todas las opciones posibles de la web

# driver.maximize_window()

# sleep(5)


def llamar_info_peli(lista_id_muestra,driver,archivo_csv):
  # Creo lista de tuplas tal y como pide el enunciado para guardar los datos extraidos 

  lista_tuplas_pelis=[]

  # Creamos una muestras de id de peliculas por las que vamos a iterar para extraer la info

  #lista_id_muestra= ['tt10579986','tt10597228','tt11097384','tt11152168','tt11237950']


  for id in lista_id_muestra:
      
      # Creo la lista donde voy a ir guardando la info de cada peli, que despues convertiremos en tupla 

      lista_datos_imdb=[]

      try: 

        sleep(3)

        busqueda = driver.find_element("css selector","#suggestion-search")

        busqueda.clear()

        busqueda.send_keys(id, Keys.ENTER)

        # Extraemos la informacion que necesitamos

        try:

          nombre_pelicula=driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div[1]/h1/span').text
          #nombre_pelicula=driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(5) > section > section > div.sc-92625f35-3.frxYSZ > div.sc-b7c53eda-0.dUpRPQ > h1 > span").text 
          lista_datos_imdb.append(nombre_pelicula)

        except:
          go_homepage= driver.find_element("xpath",'//*[@id="__next"]/main/div[2]/div/div[2]/div/div[1]/a').click()
          lista_datos_imdb.append("No encontrada")
          print("No se ha encontrado esta película en IMDB")


        # DIRECCION: extraemos la informacion de los directores, que pueden ser 1 o varios
        # Creamos una lista donde iremos agregando los diferentes directores en base a las etiquetas correspondientes

        #direccion_peli=[]

        try: 
          
          director1= driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[2]/ul/li[1]/div/ul/li[1]/a').text
          lista_datos_imdb.append(director1) 

        except:  

            try:
                   
              director1= driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[3]/ul/li[1]/div/ul/li/a').text
              lista_datos_imdb.append(director1)

            except:
               
               director2= driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[4]/ul/li[1]/div').text
               lista_datos_imdb.append(director2)     
                                            
        # Guionistas

        try:
          
          guionistas_peli= driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[2]/ul/li[2]/div').text
          guionistas_bien=re.sub(r'([a-z])([A-Z])', r'\1 \2', guionistas_peli)
          lista_datos_imdb.append(guionistas_bien)

        except:
            try:
              guionistas_peli= driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[3]/ul/li[2]/div').text
              guionistas_bien=re.sub(r'([a-z])([A-Z])', r'\1 \2', guionistas_peli)
              lista_datos_imdb.append(guionistas_bien)

            except:
                guionistas_peli=driver.find_element("xpath",'//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[4]/ul/li[2]/div').text
                guionistas_bien=re.sub(r'([a-z])([A-Z])', r'\1 \2', guionistas_peli)
                lista_datos_imdb.append(guionistas_bien) 


        # Argumento
        # Para argumento hay varias etiquetas, una para cada tamaño de pantalla o dispositivo, por lo que hay que escoger la de pantalla grande
        
        argumento_peli=driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/p/span[3]').text
        lista_datos_imdb.append(argumento_peli) 

      
        # Duracion pelicula:

        try: 
          duracion_peli=driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div[1]/ul/li[3]').text
          #duracion_peli= driver.find_element("css selector","__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(28) > div.sc-f65f65be-0.bBlII > ul > li:nth-child(1) > div").text
          lista_datos_imdb.append(duracion_peli)

        except:

          try:  
            duracion_peli=driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div[1]/ul/li[2]').text
            lista_datos_imdb.append(duracion_peli)
            
          except:
            lista_datos_imdb.append("No encontrado")
          
    
        # Puntuacion
        try:
          
          puntuacion_imdb= driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div[2]/div/div[1]/a/span/div/div[2]/div[1]/span[1]').text
          lista_datos_imdb.append(puntuacion_imdb.replace(",", "."))

        except:
        
          lista_datos_imdb.append("No encontrado")

      except Exception as null:
          print(f"Error al procesar la búsqueda para el ID {id}: {str(null)}")

    
    # Guardo todos los datos extraidos de cada peli en una tupla, que se incluye, a su vez, en la lista de tuplas de todas las peliculas

      lista_tuplas_pelis.append(tuple(lista_datos_imdb))

   # Guardo informacion en un csv

  with open(archivo_csv, 'w', newline='', encoding='utf-8') as archivo:
         
     escritor =csv.writer(archivo) 
     escritor.writerow(["Nombre", "Director", "Guionista", "Argumento", "Duración", "Puntuación IMDb"])
     escritor.writerows(lista_tuplas_pelis)
  
   # Sale a pagina principal y vuelve a empezar bucle

  sleep(3)

  driver.back() 

  sleep(3)

  return lista_tuplas_pelis

In [26]:
#lista_tuplas_pelis

[("Mom's Coming",
  'Rishiking',
  'Rishiking',
  'Un alma supuestamente tiene como hija a una chica de 20 años, a la que quiere llevarse con ella, y la hermana mayor quiere impedir que el alma lo haga.',
  '2h 2min',
  'No encontrado'),
 ('Love Song and Power',
  'Erik Krefeld',
  'Stevie Kinchen Erik Krefeld',
  'Esta es una historia sobre encontrar el valor para ser uno mismo.',
  '1h 30min',
  'No encontrado'),
 ('El astronauta',
  'Johan Renck',
  'Jaroslav Kalfar Colby Day',
  'Jakub Procházka, huérfano y criado en el campo por sus abuelos, supera todas las adversidades y se convierte en el primer astronauta del país.',
  '1h 47min',
  '5.7'),
 ('Amigos imaginarios',
  'John Krasinski',
  'John Krasinski',
  'Sigue a una niña que pasa por una experiencia difícil y entonces empieza a ver a los amigos imaginarios de todo el mundo que se han quedado atrás cuando sus amigos de la vida real han crecido.',
  '1h 44min',
  '8.0'),
 ('The Accompanist Awakening',
  'Frederick Keeve',
  'F

In [6]:
#resultados_muestra

[("Mom's Coming",
  'Rishiking',
  'Rishiking',
  'Un alma supuestamente tiene como hija a una chica de 20 años, a la que quiere llevarse con ella, y la hermana mayor quiere impedir que el alma lo haga.',
  '2h 2min',
  'No encontrado'),
 ('Love Song and Power',
  'Erik Krefeld',
  'Stevie Kinchen Erik Krefeld',
  'Esta es una historia sobre encontrar el valor para ser uno mismo.',
  '1h 30min',
  'No encontrado'),
 ('El astronauta',
  'Johan Renck',
  'Jaroslav Kalfar Colby Day',
  'Jakub Procházka, huérfano y criado en el campo por sus abuelos, supera todas las adversidades y se convierte en el primer astronauta del país.',
  '1h 47min',
  '5.7'),
 ('Amigos imaginarios',
  'John Krasinski',
  'John Krasinski',
  'Sigue a una niña que pasa por una experiencia difícil y entonces empieza a ver a los amigos imaginarios de todo el mundo que se han quedado atrás cuando sus amigos de la vida real han crecido.',
  '1h 44min',
  '7.6'),
 ('The Accompanist Awakening',
  'Frederick Keeve',
  'F


Extraemos de los CSV de la API las listas por id

In [7]:
# Importamos el csv de Accion para extraer los ID y llamar a la funcion

datos_accion_csv=pd.read_csv(r"C:\Users\silvi\Desktop\Adalab\project-da-promo-H-module-2-team-Interstellar\API\CSVs\Películas_['Action']_2010-2024.csv") 
datos_comedia_csv=pd.read_csv(r"C:\Users\silvi\Desktop\Adalab\project-da-promo-H-module-2-team-Interstellar\API\CSVs\Películas_['Comedy']_2010-2025.csv")
datos_drama_csv=pd.read_csv(r"C:\Users\silvi\Desktop\Adalab\project-da-promo-H-module-2-team-Interstellar\API\CSVs\Películas_['Drama']_2010-2025.csv")

In [8]:
type(datos_accion_csv)

pandas.core.frame.DataFrame

In [9]:
# Quiero ver como se llaman las columnas del csv para buscar la columna de ID

print(datos_comedia_csv.columns)

Index(['Tipo', 'Género', 'Título', 'Mes', 'Año', 'ID_IMDB'], dtype='object')


In [8]:
# Extraemos los id y posteriormente los convertiremos a una lista

id_accion=datos_accion_csv["ID_IMDB"]
id_comedia=datos_comedia_csv["ID_IMDB"]
id_drama=datos_drama_csv["ID_IMDB"]

In [9]:
lista_id_accion=id_accion.tolist()
lista_id_comedia=id_comedia.tolist()
lista_id_drama=id_drama.tolist()

In [10]:
#  todos los id en una lista 

union_listas =[]
union_listas.extend(lista_id_accion)
union_listas.extend(lista_id_comedia)
union_listas.extend(lista_id_drama)

In [11]:
# cuento numero de elementos en la lista
print(len(union_listas))

15688


In [12]:
# elimino id duplicados duplicados

lista_id_set = set(union_listas)
print(len(lista_id_set))

14544


In [13]:
#lo convierto en lista

lista_id = list(lista_id_set)
len(lista_id)

14544

In [57]:
lista_id.index("tt10022048")

894

Creamos sublistas para ir ejecutando poco a poco

In [47]:
sublista_1=lista_id[:1263] 
len(sublista_1) 


1263

In [31]:
sublista_2=lista_id[1263:2500] 
len(sublista_2)

1237

In [32]:
sublista_3=lista_id[2500:3750] 
len(sublista_3)

1250

In [33]:
# sublista_4=lista_id[3750:5000] 
# len(sublista_4)

1250

In [34]:
# sublista_5=lista_id[5000:6250] 
# len(sublista_5)

1250

In [35]:
# sublista_6=lista_id[6250:7500] 
# len(sublista_6)

1250

In [36]:
# sublista_7=lista_id[7500:8750] 
# len(sublista_7)

1250

In [37]:
# sublista_8=lista_id[8750:10000] 
# len(sublista_8)

1250

In [38]:
# sublista_9=lista_id[10000:11250] 
# len(sublista_9)

1250

In [41]:
# sublista_10=lista_id[11250:12500] 
# len(sublista_10)

1250

In [45]:
# sublista_11=lista_id[12500:13750] 
# len(sublista_11)

1250

In [46]:
# sublista_12=lista_id[13750:-1] 
# len(sublista_12)

793

Llamamos a la función

In [50]:
# Llamo al navegador

driver=webdriver.Chrome()
driver.get("https://www.imdb.com/?ref_=nv_home")
driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()

# Hago grande el navegador para que aparezcan todas las opciones posibles de la web

driver.maximize_window()

sleep(5) 

# Lista de "id" por la que quiero iterar dentro de la funcion

######lista_id_muestra= ['tt10579986','tt10597228','tt11097384','tt11152168','tt11237950'] 

#lista_muestraaccion=['tt0411951','tt0423474','tt20877586','tt0429493','tt0439801','tt0446029','tt0464162','tt0473075','tt0477080'] 

# Creo un archivo csv para agregar la informacion

csv_info_2 = 'resultados_info_sublista2.csv'

# Llamo a la funcion 

resultados_muestra=llamar_info_peli(sublista_1,driver,csv_info_2)

driver.close()

Error al procesar la búsqueda para el ID tt10021758: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[4]/ul/li[2]/div"}
  (Session info: chrome=124.0.6367.201); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF748691562+60802]
	(No symbol) [0x00007FF74860AC62]
	(No symbol) [0x00007FF7484C7CE4]
	(No symbol) [0x00007FF748516D4D]
	(No symbol) [0x00007FF748516E1C]
	(No symbol) [0x00007FF74855CE37]
	(No symbol) [0x00007FF74853ABBF]
	(No symbol) [0x00007FF74855A224]
	(No symbol) [0x00007FF74853A923]
	(No symbol) [0x00007FF748508FEC]
	(No symbol) [0x00007FF748509C21]
	GetHandleVerifier [0x00007FF7489941FD+3217949]
	GetHandleVerifier [0x00007FF7489D6197+3488183]
	GetHandleVerifier [0x00007FF7489CF11F+3459391]
	GetHandleVerifier [0x00007FF74874B926+823622]
	(N

In [59]:
resultados_muestra

[('Mazhor. Film',
  'Nikolai Bulygin',
  'Sergey Kaluzhanov Oleg Malovichko',
  'Not even a bullet can stop a real hero. Full-length sequel to the high-profile series with Pavel Prilunny',
  '1h 38min',
  '5.3'),
 ('Klovn the Final',
  'Mikkel Nørgaard',
  'Casper Christensen Frank Hvam Mikkel Nørgaard',
  "Frank turns 50 and his good friend, Casper, invites him to Iceland on a boys' trip. However, the trip does not go quite as planned.",
  '1h 34min',
  '6.6'),
 ('Let Me Help Out', 'Kyanna Mai-Lee Bosworth'),
 ('Kazhdyy mechtaet o sobake',
  'Olga Belyaeva',
  'Olga Belyaeva',
  "Based on the story of the same name by Vladimir Zheleznikov. The film's narrative will unfold in 1965 in Moscow.",
  '1h 30min',
  'No encontrado'),
 ('One Million K(l)icks',
  'Alex Padrutt',
  'Mike Möller Marco Theiss',
  'Un joven luchador con un noble corazón se ve obligado a participar en un oscuro torneo de artes marciales organizado en Internet.',
  '1h 38min',
  '4.5'),
 ('Nose Candy', 'Lydia Martine

In [60]:
len(resultados_muestra)

1263

Puntos de errores 

Para saber donde se ha quedado parado el codigo tengo que buscar index en la lista general,  no en la sublista!! 

In [52]:
sublista_2

['tt10221766',
 'tt2231489',
 'tt2027215',
 'tt1109488',
 'tt14298296',
 'tt10476740',
 'tt1331329',
 'tt2145748',
 'tt10109870',
 'tt12751534',
 'tt10219112',
 'tt10508774',
 'tt10125766',
 'tt0446029',
 'tt3863892',
 'tt13451738',
 'tt10850600',
 'tt2676350',
 'tt10091230',
 'tt10048426',
 'tt11718134',
 'tt2509902',
 'tt12754776',
 'tt13612924',
 'tt10073628',
 'tt1611823',
 'tt2271563',
 'tt1893241',
 'tt10066102',
 'tt15691864',
 'tt22941760',
 'tt2629524',
 'tt10006276',
 'tt16148626',
 'tt10713294',
 'tt29940843',
 'tt1803745',
 'tt1929344',
 'tt13445076',
 'tt10960402',
 'tt10515720',
 'tt2250690',
 'tt3370448',
 'tt10122106',
 'tt10163256',
 'tt10964096',
 'tt13053972',
 'tt1600057',
 'tt2206082',
 'tt10309902',
 'tt10692870',
 'tt11002292',
 'tt10217030',
 'tt24652070',
 'tt2387395',
 'tt11756362',
 'tt10666600',
 'tt1242521',
 'tt12917262',
 'tt10144414',
 'tt14809390',
 'tt11355288',
 'tt10891822',
 'tt32122735',
 'tt15145972',
 'tt10081162',
 'tt10461442',
 'tt12991814',
 

In [58]:
sublista_2[894]

'tt11332850'

In [55]:
sublista_2.index("")

896